<a href="https://colab.research.google.com/github/ktpeters/Calculating-Vegetation-Indices-from-Planet-API/blob/main/Landsat_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install earthengine-api

In [ ]:
import ee
import os
import time

In [ ]:
# # Get the path to the credentials file
credentials_path = os.path.expanduser("~/.config/earthengine/credentials")

# # Remove the credentials file if it exists
if os.path.exists(credentials_path):
     os.remove(credentials_path)
     print("Earth Engine credentials removed. You are now unauthenticated.")
else:
  print("No Earth Engine credentials found.")

No Earth Engine credentials found.


In [ ]:
# Trigger the authentication flow. You only need to do this once.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-covercropproject')

In [ ]:
# Define the FeatureCollection
state_abb = 'MO'
polygons = ee.FeatureCollection(f"projects/ee-covercropproject/assets/CLU_{state_abb}")
#projects/ee-indianproject/assets/CLU_IL



# Sort the FeatureCollection by a specific property before converting to a list
sortedCollection = polygons.sort('ID')

featureList = sortedCollection.toList(sortedCollection.size())
#print(featureList)

In [ ]:
#print(sortedCollection.size())
print(sortedCollection.size().getInfo())

467266


In [ ]:
# Define a function to extract the relevant bits and create a mask
def mask_clouds(image):
    # Select the SR_CLOUD_QA band
    cloud_qa = image.select('SR_CLOUD_QA')

    # Define bitmasks for the relevant bits
    cloud_bitmask = 1 << 1  # Bit 1 for high confidence cloud
    shadow_bitmask = 1 << 3  # Bit 3 for high confidence cloud shadow

    # Identify the pixels with clouds or cloud shadows
    cloud_mask = cloud_qa.bitwiseAnd(cloud_bitmask).eq(0)
    shadow_mask = cloud_qa.bitwiseAnd(shadow_bitmask).eq(0)

    # Combine the masks
    combined_mask = cloud_mask.And(shadow_mask)

    # Apply the mask to the image
    return image.updateMask(combined_mask)

# Function to calculate various indices and add date
def addNDVInDate(image):
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    tvi = image.expression(
        '(pow((((B8-B4)/(B8+B4)) +(1/2)),(1/2)))*100', {
            'B8': image.select('SR_B4'),
            'B4': image.select('SR_B3')
        }).rename('TVI')
    evi = image.expression(
        '2.5* ( (B8-B4)/ (B8+ 6*B4 - 7.5*B2 +1))', {
            'B4': image.select('SR_B3'),
            'B8': image.select('SR_B4'),
            'B2': image.select('SR_B1')
        }).rename('EVI')
    satvi = image.expression(
        '((B11-B4)/(B11+B4+1)) * (1+1) - (B12/2)', {
            'B11': image.select('SR_B5'),
            'B4': image.select('SR_B3'),
            'B12': image.select('SR_B7')
        }).rename('SATVI')
    savi = image.expression(
        '((B8-B4) + (1+0.5))/ (B8-B4+0.5)', {
            'B4': image.select('SR_B3'),
            'B8': image.select('SR_B4')
        }).rename('SAVI')
    msi = image.expression(
        '(B11/B8)', {
            'B11': image.select('SR_B5'),
            'B8': image.select('SR_B4')
        }).rename('MSI')
    gndvi = image.normalizedDifference(['SR_B4', 'SR_B2']).rename('GNDVI')
    grvi = image.normalizedDifference(['SR_B2', 'SR_B3']).rename('GRVI')
    lswi = image.normalizedDifference(['SR_B4', 'SR_B5']).rename('LSWI')
    msavi2 = image.expression(
        '(2 * B5 + 1 - sqrt(pow((2 * B5 + 1), 2) - 8 * (B5 - B4)) ) / 2', {
            'B5': image.select('SR_B4'),
            'B4': image.select('SR_B3')
        }).rename('MSAVI2')
    wdvi = image.expression(
        'B8- 0.5*B4', {
            'B4': image.select('SR_B3'),
            'B8': image.select('SR_B4')
        }).rename('WDVI')
    bi = image.expression(
        'sqrt((B4*B4) +(B3*B3))/2', {
            'B4': image.select('SR_B3'),
            'B3': image.select('SR_B2')
        }).rename('BI')
    bi2 = image.expression(
        'sqrt((B4*B4) +(B3*B3)+(B8*B8))/3', {
            'B4': image.select('SR_B3'),
            'B3': image.select('SR_B2'),
            'B8': image.select('SR_B4')
        }).rename('BI2')
    ri = image.expression(
        '(B4*B4)/(B3*B3*B3)', {
            'B4': image.select('SR_B3'),
            'B3': image.select('SR_B2')
        }).rename('RI')
    ci = image.normalizedDifference(['SR_B3', 'SR_B2']).rename('CI')
    v = image.expression(
        '(B8/B4)', {
            'B8': image.select('SR_B4'),
            'B4': image.select('SR_B3')
        }).rename('V')
    ndwi = image.normalizedDifference(['SR_B4', 'SR_B5']).rename('NDWI')
    nbr = image.normalizedDifference(['SR_B4', 'SR_B7']).rename('NBR')

    newBands = ee.Image([ndvi, tvi, evi, satvi, savi, msi, gndvi, grvi, lswi,
                         msavi2, wdvi, bi, bi2, ri, ci, v, ndwi, nbr])
    return image.addBands(newBands)

# Function to reduce the regions and calculate statistics
def reduceRegionsFunction(image):
    stats = image.reduceRegions(
        collection=subsetFC,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    stats_filtered = stats.filter(ee.Filter.notNull(['NDVI']))
    return stats_filtered.map(lambda feature: feature.set('date', image.get('system:time_start')).setGeometry(None))

In [ ]:
# Iterate through the FeatureCollection in chunks
#time.sleep(10800)
stepsize=2000
for i in range(0, sortedCollection.size().getInfo()+1, stepsize):
    subset = featureList.slice(i, i+stepsize)
    subsetFC = ee.FeatureCollection(subset)
    convexHull = subsetFC.geometry().convexHull().buffer(50)

    # Download NDVI data for each year
    for yr in range(2015, 2025):
        startDate = f'{yr-1}-05-15'
        endDate = f'{yr}-5-15'
        fileName = f'NDVI_{yr-1}_{i}'

        landsat = (ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
                 .filterBounds(convexHull)
                 .filterDate(startDate, endDate)
                 .map(mask_clouds)
                 .map(addNDVInDate))

        reduced = landsat.map(reduceRegionsFunction).flatten()
        # Remove geometry before exporting
        reduced_no_geom = reduced.map(lambda feature: feature.setGeometry(None))

        # Export the data to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=reduced_no_geom,
            description=fileName,
            folder=f'Landsat7_CLU_{state_abb}_sorted',
            fileFormat='CSV'
        )
        task.start()